In [3]:
!pip install pennylane

In [21]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt


def encode_data(weather):
    sunny_condition = np.all(weather == "sunny")
    rainy_condition = np.all(weather == "rainy")

    if sunny_condition:
        return 0
    elif rainy_condition:
        return 2 * np.pi / 3
    else:
        return 4 * np.pi / 3



# Quantum Node: QML circuit
@qml.qnode(dev)
def weather_circuit(weather_data, params):
    for i in range(len(weather_data)):
        qml.RX(encode_data(weather_data[i]), wires=0)
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))


def cost(params, data):
    total_loss = 0
    for sequence in data:
        weather_data, actual_weather = sequence[:-1], sequence[-1]
        predicted_val = weather_circuit(weather_data, params)
        actual_val = encode_data(actual_weather) / (2 * np.pi) * 3
        total_loss += (predicted_val - actual_val) ** 2  # MSE
    return total_loss / len(data)


# Quantum Node for visualization purposes
def visualize_circuit(weather_data, params):
    for i in range(len(weather_data)):
        qml.RX(encode_data(weather_data[i]), wires=0)
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))


# Training data
def generate_random_data(num_samples):
    weather_types = ["sunny", "rainy", "cloudy"]
    data = []
    for _ in range(num_samples):
        sequence = np.random.choice(weather_types, size=7)
        actual_weather = np.random.choice(weather_types)
        data.append((sequence, actual_weather))
    return data


# Prediction and decoding
def predict(weather_data, trained_params):
    prediction_val = weather_circuit(weather_data, trained_params)
    if prediction_val < 1 / 3:
        return "sunny"
    elif prediction_val < 2 / 3:
        return "rainy"
    else:
        return "cloudy"


if __name__ == '__main__':
    # Quantum device
    dev = qml.device("default.qubit", wires=1)

    params = np.random.random(2)  # Initialize parameters randomly
    opt = qml.GradientDescentOptimizer(stepsize=0.1)

    # Generate random training data
    training_data = generate_random_data(num_samples=100)

    # Training loop
    epochs = 100
    for epoch in range(epochs):
        params = opt.step(lambda p: cost(p, training_data), params)

    # Display the predicted weather for a random sequence of the last 7 days
    random_weather_sequence = np.random.choice(["sunny", "rainy", "cloudy"], size=7)
    print(f"Predicted Weather: {predict(random_weather_sequence, params)}")

    # Display the trained circuit for the random sequence
    drawn_circuit = qml.draw(visualize_circuit)
    print(drawn_circuit(random_weather_sequence, params))


Predicted Weather: cloudy
0: ──RX(2.09)──RX(2.09)──RX(4.19)──RX(4.19)──RX(0.00)──RX(0.00)──RX(4.19)──RX(1.94)──RY(0.04)─┤  <Z>
